In [ ]:
## question 3
#a
df1 = pd.read_sas('DEMO_G.XPT')
df2 = pd.read_sas('DEMO_H.XPT')
df3 = pd.read_sas('DEMO_I.XPT')
df4 = pd.read_sas('DEMO_J.XPT')
df1['cohort_number'] = '2011-2012'
df2['cohort_number'] = '2013-2014'
df3['cohort_number'] = '2015-2016'
df4['cohort_number'] = '2017-2018'
col = ['SEQN','RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR','SDMVPSU','SDMVSTRA',
       'WTINT2YR','WTMEC2YR','cohort_number']
print(df1[col].head())
df = pd.concat([df1,df2,df3,df4], ignore_index=True)[col]
df.rename(columns = {'SEQN':'ID', 'RIDAGEYR':'Age', 'RIDRETH3': 'Race',
                    'DMDEDUC2':'education', 'DMDMARTL': 'marital_status',
                    'RIDSTATR':'Interview&examination_status', 'SDMVPSU':'pseudo-PSU-var',
                    'SDMVSTRA': 'pseudo-stratum-var', 'WTMEC2YR': 'interview&examined',
                    'WTINT2YR': 'interviewed'},inplace = True)
# Transform column names to lower case
df.columns = df.columns.str.lower()
# Convert to appropriate dtype
df = df.convert_dtypes()
# drop anomilies and conv
df = df.query('age >= 1')
df['age'] = df['age'].astype('int64')
df['race'] = df['race'].astype('category')
df['marital_status'] = df['marital_status'].astype('category')
df['interview&examination_status'] = df['interview&examination_status'].astype('category')
# checking if row wise elements are all null
def check_null(df):
    """
    

    Parameters
    ----------
    df : Dataframe
        original dataframe

    Returns
    -------
    Datefreame without null rows

    """
    is_Nan = df.isnull()
    row_Nan = is_Nan.all(axis = 1) 
    index_lst = df[row_Nan].index
    new_df = df.drop(index_lst)
    if df.shape == new_df.shape:
        print('There is NO invalid rows with all nan values in the dataset')
    return new_df
df = check_null(df)
# save to pickle
df.to_pickle('demographics_saved_as_pickle')
#b
total_file = os.listdir()
pattern = re.compile("OHXDEN_\S+") 
total_name = ''
for file in total_file:
    total_name += file + ' '
file_lst = pattern.findall(total_name)
# print(file_lst)
lst_year = ['2011-2012','2013-2014','2015-2016','2017-2018']
dic = dict(zip(file_lst,lst_year))
df_b = pd.DataFrame()
for file,year in dic.items():
    temp = pd.read_sas(file)
    temp['cohort_number'] = dic[file]
    df_b = pd.concat([temp, df_b], ignore_index = True)
# print(df_b.columns)
# creating pattern matching "Tooth count" and "coronal cavities"
pattern = re.compile("OHX.{2}TC|OHX.{2}CTC")
col_lists = [col for col in df_b.columns]
total_name = ''
# generating a string that caontains matching col names
for name in col_lists:
    total_name += name + ' '
col_lst = pattern.findall(total_name)
col = ['SEQN','OHDDESTS'] + col_lst
df_short = df_b[col].rename(columns={'SEQN':'ID', 'OHDDESTS':'Dentition_Status'})
# use check_null function to check null rows
df_short = check_null(df_short)
# convert column names to lower case
df_short.columns = df_short.columns.str.lower()
def convert_names(df):
    """
    Convert names to tooth count and coronal cavities plus numbers

    Parameters
    ----------
    df : Dataframe
        priginal Dataframe

    Returns
    -------
    df : Dataframe
        new Dataframe with edited column names

    """
    lst = []
    col = df.columns[2:]
    for name in col:
        if len(name) == 7:
            name = re.sub(r'OHX(.{2})TC' , r'\1tooth_count',name)
        else:
            name = re.sub(r'OHX(.{2})CTC' , r'\1coronal_cavities',name)
        lst.append(name)
    new_list = list(df.columns[:2]) + lst
    df.columns = new_list
    return df
# call convert_name function 
df_short = convert_names(df_short)
# convert to pickle
df_short.to_pickle('dentition_saved_as_pickle')
#c
print('Dataframe size of question a is', df.shape,'\n'
      'Dataframe size of question b  is', df_short.shape)